In [ ]:
import unittest
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sqlalchemy
import matplotlib.pyplot as plt

# Import data
test = pd.read_csv('C:\\Users\\DMA Analytics\\Desktop\\Masters\\Python\\Datasets1\\test.csv')
train = pd.read_csv('C:\\Users\\DMA Analytics\\Desktop\\Masters\\Python\\Datasets1\\train.csv')
ideal = pd.read_csv('C:\\Users\\DMA Analytics\\Desktop\\Masters\\Python\\Datasets1\\ideal.csv')

class TestIdealFunctions(unittest.TestCase):
    def setUp(self):
        self.test = pd.read_csv('C:\\Users\\DMA Analytics\\Desktop\\Masters\\Python\\Datasets1\\test.csv')
        self.train = pd.read_csv('C:\\Users\\DMA Analytics\\Desktop\\Masters\\Python\\Datasets1\\train.csv')
        self.ideal = pd.read_csv('C:\\Users\\DMA Analytics\\Desktop\\Masters\\Python\\Datasets1\\ideal.csv')

    def test_best_fit_selection(self):
        mses = {}
        for y in self.train:
            if y != 'x':
                errors = {}
                for x in self.ideal:
                    if x != 'x':
                        Y = self.train[y]
                        X = self.ideal[x]
                        X = sm.add_constant(X)
                        model = sm.OLS(Y, X).fit()
                        errors[x] = model.mse_resid
                mses[y] = min(errors, key=errors.get)
        self.assertTrue(isinstance(mses, dict))

    def test_maximum_deviation_calculation(self):
        mdev = {}
        select = self.train.merge(self.ideal[[y for x in [list(mses.values()), 'x'] for y in x]], on='x', how='left')
        for item in mses:
            select[item + '_' + mses[item]] = select[item] - select[mses[item]]
            mdev[mses[item]] = select[item + '_' + mses[item]].max() * np.sqrt(2)
        self.assertTrue(isinstance(mdev, dict))

    def test_compliance_check(self):
        tested = self.test.merge(self.ideal[[y for x in [list(mses.values()), 'x'] for y in x]], on='x', how='left')
        devs = {}
        for y in tested:
            if (y != 'x') & (y != 'y'):
                devs[y] = np.diff(tested[[y, 'y']], axis=1).max()
        comply = {}
        for y in devs:
            if devs[y] <= mdev[y]:
                comply[y] = devs[y]
        self.assertTrue(isinstance(comply, dict))

    def test_database_export(self):
        engine = sqlalchemy.create_engine('sqlite:///C:\\Users\\DMA Analytics\\Desktop\\Masters\\Python\\sqlite\\pythonsqlite.db', echo=False)
        self.train.to_sql('train', con=engine, if_exists='append')
        self.ideal.to_sql('ideal', con=engine, if_exists='append')
        self.tested.to_sql('tested', con=engine, if_exists='append')
        with engine.connect() as con:
            result = con.execute("SELECT count(*) FROM train")
            train_count = result.scalar()
            result = con.execute("SELECT count(*) FROM ideal")
            ideal_count = result.scalar()
            result = con.execute("SELECT count(*) FROM tested")
            tested_count = result.scalar()
        self.assertNotEqual(train_count, 0)
        self.assertNotEqual(ideal_count, 0)
        self.assertNotEqual(tested_count, 0)

    def tearDown(self):
        del self.test
        del self.train
        del self.ideal

if __name__ == '__main__':
    unittest.main()